In [1]:
LANGCHAIN_TRACING_V2=True

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import getpass, os
from dotenv import load_dotenv
load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
search.invoke("what is the weather in SF")

In [ ]:
retriever.invoke("how to upload a dataset")[0]

In [ ]:
from langchain.tools.retriever import create_retriever_tool

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [ ]:
tools = [search, retriever_tool]

create the agent

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel


vertexai.init(project=os.getenv('PROJECT_ID'), location=os.getenv('LOCATION'))

model = GenerativeModel("gemini-1.5-pro-001")
response = model.generate_content(
    "generate a bear surfing on space"
)

print(response.text)

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro")
llm.invoke("Sing a ballad of LangChain.")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI

system = "You are a helpful assistant who translate English to French"
human = "Translate this sentence from English to French. I love programming."
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chat = ChatVertexAI()

chain = prompt | chat
chain.invoke({})

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
search  = TavilySearchResults()
search.invoke("what is the weather in SF")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("Write a ballad about LangChain")
print(result.content)